In [6]:
import requests, json
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import plotly.express as px

class CONFIG:
    def __init__(self) -> None:
        self.group_id = 15130

In [7]:
def get_users(group_id):
    skcn_ranklist = f"https://www.acmicpc.net/group/ranklist/{group_id}"
    resp = requests.get(url=skcn_ranklist)
    bs = BeautifulSoup(resp.text,"html.parser")
    # ranklist > tbody > tr:nth-child(1) > td:nth-child(2)
    bs_users = bs.select("#ranklist > tbody > tr > td:nth-child(2)")
    users = []
    for u in bs_users:
        users.append(u.text)
    return users
# users = get_users()

In [8]:
# https://solved.ac/api/v3/user/show
def get_rating(handle:str):
    url = "https://solved.ac/api/v3"
    headers = { 'Content-Type': "application/json" }
    params = {'handle':handle}
    resp = requests.get(url+"/user/show",params=params,headers=headers)
    me = json.loads(resp.text)
    return (me['rating'])

def get_user_history(handle:str)->list:
    url = f"https://solved.ac/profile/{handle}/history"
    resp = requests.get(url)
    bs = BeautifulSoup(resp.text,"html.parser")
    raw_data = bs.select("#__NEXT_DATA__")[0].text
    data = json.loads(raw_data)
    hist = data['props']['pageProps']['histories']
    rating,rank,exp,solved_cnt,contrib_cnt = hist
    return hist

In [9]:
def get_user_lang_stats(handle:str):
    # print("get_user_lang_stats: ",handle)
    url = f"https://www.acmicpc.net/user/language/{handle}"
    cols = ["lang","ac","stouterr","wa","tle","mem","writeover","runtime","compileerr","_","ac_problems","submit","ratio"]
    df = pd.read_html(url, encoding="utf-8")[0]
    df.columns = cols
    df.fillna(int(0),inplace=True)
    return df

def aggr_lang_stats():
    ans = get_user_lang_stats(users[0])
    for user in users[1:]:
        ans.add(get_user_lang_stats(user))
    return ans

In [10]:
def plot_user_solved(handle:str):
    data = get_user_history(handle)[-2] #,columns=["timestamp","value"]
    df = pd.DataFrame.from_records(data,columns=['timestamp','value'])
    # fig = px.scatter(df)
    fig = px.line(df,x=df['timestamp'],y=df['value'])
    fig.show()

def aggr_users_solved(group_id):
    users = get_users(group_id)
    data = get_user_history(users[0])[-2]
    solved_df = pd.DataFrame.from_records(data,columns=['timestamp','value'])
    for user in users[1:]:
        data = get_user_history(user)[-2]
        df = pd.DataFrame.from_records(data,columns=['timestamp','value'])
        solved_df[user] = df['value']
    solved_df.fillna(0,inplace=True)
    return solved_df

df = aggr_users_solved(15130)
fig = px.line(df,x=df['timestamp'],y=df.columns)
fig.show()